In [172]:
print('hello')

hello


In [173]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

True

In [174]:
from langchain_openai import ChatOpenAI
from langsmith import traceable
# Initialize ChatOpenAI with LM Studio endpoint
llm = ChatOpenAI(
    base_url="http://172.17.160.1:1234/v1",  # Can be any dummy value
    api_key='None',
    model="gemma-3-4b-instruct",  # Optional
    temperature=0.7
)

# Use the model
response = llm.invoke("Hello, how are you?")
print(response.content)

I’m doing well, thank you for asking! As an AI, I don't really *feel* in the same way humans do, but my systems are running smoothly and I’m ready to chat. 😊 

How about you? How is your day going so far? Do you want to talk about anything specific, or just have a general conversation?


In [175]:
from openai import OpenAI

In [176]:
openai_client = OpenAI(api_key = 'lmstudio',base_url='http://172.17.160.1:1234')

In [177]:
import requests

# Test basic connectivity
try:
    response = requests.get('http://172.17.160.1:1234/v1/models')
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Connection Error: {e}")


Status Code: 200
Response: {'data': [{'id': 'gemma-3-4b-it', 'object': 'model', 'owned_by': 'organization_owner'}, {'id': 'text-embedding-nomic-embed-text-v1.5', 'object': 'model', 'owned_by': 'organization_owner'}], 'object': 'list'}


In [178]:
@traceable()
def retriever(query: str):
    if 'ishita' in query:
        results = ["Ishita and akshat work as interns at artista , both are fat and methodical in their workfdjhfnkdfbdhjkfbvdjfhvdfhdvbfdjmfbdmfjbdjfbdfjbdkjfdbfkdbfhjkdbf"]
    else:
        return 'Akshat is fat'
    return results

# This is the end-to-end RAG chain.
# It does a retrieval step then calls OpenAI
@traceable()
def rag(question):
    docs = retriever(question)
    print(docs)
    system_message = """Answer the users question using only the provided information below: {docs}""".format(docs="\n".join(docs))
    llm_resp = llm.invoke(f"{system_message} \n {question}")
    return llm_resp.content

In [179]:
rag('Akshat is fat?')

Akshat is fat


"The text provides letters: A, k, s, h, a, t, i, s, f, a, t. It also states that Akshat is fat. However, it doesn't give any information about whether *Akshat* specifically is fat."

In [180]:
from langgraph.prebuilt import create_react_agent

@traceable()
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}"

@traceable()
def get_personal_info(query : str):
    ''' Gets Personal info of the model developer'''
    return f'Name is akshat khatri and 20 years old'

agent = create_react_agent(
    model=llm,
    tools=[get_weather,get_personal_info],
    prompt="You are a helpful assistant , and help me",
)
# Run the agent
resp = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco? and who is model's developer"}]}
)

for msg in resp['messages']:
    print(msg.content)

What is the weather in San Francisco? and who is model's developer

It's always sunny in San Francisco
Name is akshat khatri and 20 years old
Okay, so the weather in San Francisco is always sunny and the model’s developer is Akshat Khatri, who is 20 years old!


In [181]:
resp

{'messages': [HumanMessage(content="What is the weather in San Francisco? and who is model's developer", additional_kwargs={}, response_metadata={}, id='b6c9a0e9-b14d-41c8-b10d-27c95841ff53'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '158135545', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': '763132411', 'function': {'arguments': '{"query":"model developer"}', 'name': 'get_personal_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 542, 'total_tokens': 609, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'id': 'chatcmpl-8mlhw04n93vmv9ikaojmda', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--70877516-1116-44b0-b664-c1a10d5eda7d-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '15

In [182]:
# Pydantic is used for data validation


In [183]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage

class Question_prompt(BaseModel):
    questions : List[str] = Field(description='Validation class for prompt output validations')

structured_llm = llm.with_structured_output(Question_prompt)
system = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answered independently. \n"""

question = """What are the main components of an LLM-powered autonomous agent system?"""
questions = structured_llm.invoke([SystemMessage(content=system)]+[HumanMessage(content=question)])

In [184]:
questions.questions

['What are the core functionalities that an LLM-powered autonomous agent needs to perform its tasks?',
 'Can you categorize the different types of modules or components typically found in such a system?',
 'How does the Large Language Model (LLM) itself fit into the overall architecture, and what specific capabilities are crucial for an agent?',
 'What kind of memory mechanisms are used to enable autonomous agents to learn and retain information over time?',
 'How do LLM-powered agents interact with external tools or APIs?',
 'What role does planning and decision-making play in the operation of these agents?',
 'How is feedback incorporated into an agent’s learning process?']

In [185]:
answers = llm.invoke(questions.questions)

In [186]:
answers

AIMessage(content='Okay, let\'s break down the core functionalities, components, architecture, and mechanisms involved in LLM-powered autonomous agents. This is a rapidly evolving field, so this represents a current understanding as of late 2023/early 2024.\n\n**1. Core Functionalities – What an Agent Needs to Do:**\n\nAt the most basic level, an LLM-powered autonomous agent needs to be able to:\n\n* **Understand Instructions (Goal Decomposition):** Receive a high-level goal and break it down into smaller, actionable steps.\n* **Information Gathering:**  Access and process information relevant to its tasks – this might involve searching the web, querying databases, or accessing internal knowledge bases.\n* **Tool Use/API Interaction:** Leverage external tools (like calculators, email clients, calendars, code interpreters) to execute specific actions.\n* **Decision Making:** Select the best course of action at each step, considering context and potential outcomes.\n* **Execution & Monit

In [187]:
for answer in answers:
    print(answer,end = '\n\n\n')

('content', 'Okay, let\'s break down the core functionalities, components, architecture, and mechanisms involved in LLM-powered autonomous agents. This is a rapidly evolving field, so this represents a current understanding as of late 2023/early 2024.\n\n**1. Core Functionalities – What an Agent Needs to Do:**\n\nAt the most basic level, an LLM-powered autonomous agent needs to be able to:\n\n* **Understand Instructions (Goal Decomposition):** Receive a high-level goal and break it down into smaller, actionable steps.\n* **Information Gathering:**  Access and process information relevant to its tasks – this might involve searching the web, querying databases, or accessing internal knowledge bases.\n* **Tool Use/API Interaction:** Leverage external tools (like calculators, email clients, calendars, code interpreters) to execute specific actions.\n* **Decision Making:** Select the best course of action at each step, considering context and potential outcomes.\n* **Execution & Monitoring:

In [188]:
# Build a semantic search engine

In [189]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [190]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

106


In [191]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,chunk_overlap=200,add_start_index=True
)

all_splits = text_splitter.split_documents(docs)
all_splits

[Document(metadata={'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': '414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='FORM 10-KFORM 10-K'),
 Document(metadata={'producer': 'Wdesk Fidelity Content Translations Version 008.001.016', 'creator': 'Workiva', 'creationdate': '2023-07-20T22:09:22+00:00', 'author': 'anonymous', 'moddate': '2023-07-26T15:13:52+08:00', 'title': 'Nike 2023 Proxy', 'source': '414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'total_pages': 106, 'page': 1, 'page_label': '2', 'start_index': 0}, page_content="UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K \n(Mark One)\n☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE FISCA

In [192]:
len(all_splits),type(all_splits[0])

(501, langchain_core.documents.base.Document)

In [193]:
text_to_embed = [text.page_content for text in all_splits]
text_to_embed

['FORM 10-KFORM 10-K',
 "UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K \n(Mark One)\n☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE FISCAL YEAR ENDED MAY 31, 2023 \nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934\nFOR THE TRANSITION PERIOD FROM TO .\nCommission File No. 1-10635 \nNIKE, Inc. \n(Exact name of Registrant as specified in its charter)\nOregon 93-0584541\n(State or other jurisdiction of incorporation) (IRS Employer Identification No.)\nOne Bowerman Drive, Beaverton, Oregon 97005-6453 \n(Address of principal executive offices and zip code)\n(503) 671-6453 \n(Registrant's telephone number, including area code)\nSECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:\nClass B Common Stock NKE New York Stock Exchange\n(Title of each class) (Trading symbol) (Name of each exchange on which registered)\nSECURITIES REGISTERED PURSUANT TO SECTION 

In [194]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

embeddings = model.encode(text_to_embed)

In [195]:
embeddings.shape

(501, 768)

In [196]:
query = ['What is nike policy in the United states revenue','What is nike brand']
query_embed = model.encode(query)
query_embed.shape

(2, 768)

In [197]:
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(embeddings,query_embed)
sim_matrix

array([[0.04592033, 0.04987223],
       [0.63717383, 0.5445834 ],
       [0.20386878, 0.12016846],
       ...,
       [0.58845747, 0.467587  ],
       [0.528323  , 0.6148398 ],
       [0.44304729, 0.6218153 ]], shape=(501, 2), dtype=float32)

In [198]:
import numpy as np
k = 10
# top_k_embeddings_index =  sim_matrix.np.argmax(sim_matrix)

In [199]:
top_k_embeddings_index = np.argpartition(sim_matrix,-k,axis = 0)[-k:]
top_k_embeddings = embeddings[top_k_embeddings_index]

In [200]:
top_k_embeddings_index

array([[209, 445],
       [453, 212],
       [ 32,  47],
       [ 10, 446],
       [450,  12],
       [ 19,  13],
       [215,  14],
       [213, 333],
       [443,  10],
       [222,  11]])

In [201]:
top_k_embeddings.shape

(10, 2, 768)

In [202]:
top_k_embeddings_index.shape

(10, 2)

In [203]:
queries_augmented_generated_indices = []

for i in range(len(query)):
    query_indices = []
    for indices in top_k_embeddings_index[:,i]:
        query_indices.append(indices)
    queries_augmented_generated_indices.append(query_indices)

In [204]:
queries_augmented_generated_indices

[[np.int64(209),
  np.int64(453),
  np.int64(32),
  np.int64(10),
  np.int64(450),
  np.int64(19),
  np.int64(215),
  np.int64(213),
  np.int64(443),
  np.int64(222)],
 [np.int64(445),
  np.int64(212),
  np.int64(47),
  np.int64(446),
  np.int64(12),
  np.int64(13),
  np.int64(14),
  np.int64(333),
  np.int64(10),
  np.int64(11)]]

In [209]:
# text_to_embed = np.array(text_to_embed)
queries_augmented_generated = [[text_to_embed[i] for i in sublist] for sublist in queries_augmented_generated_indices]
queries_augmented_generated

[['CONSOLIDATED OPERATING RESULTS\nREVENUES\n(Dollars in millions)\nFISCAL \n2023\nFISCAL \n2022\n% \nCHANGE\n% CHANGE \nEXCLUDING \nCURRENCY \nCHANGES\n(1)\nFISCAL \n2021\n% \nCHANGE\n% CHANGE \nEXCLUDING \nCURRENCY \nCHANGES\n(1)\nNIKE, Inc. Revenues:\nNIKE Brand Revenues by:\nFootwear $ 33,135 $ 29,143  14 %  20 % $ 28,021  4 %  4 %\nApparel  13,843  13,567  2 %  8 %  12,865  5 %  6 %\nEquipment  1,727  1,624  6 %  13 %  1,382  18 %  18 %\nGlobal Brand Divisions(2)  58  102  -43 %  -43 %  25  308 %  302 %\nTotal NIKE Brand Revenues $ 48,763 $ 44,436  10 %  16 % $ 42,293  5 %  6 %\nConverse  2,427  2,346  3 %  8 %  2,205  6 %  7 %\nCorporate(3)  27  (72)  —  —  40  —  — \nTOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710  10 %  16 % $ 44,538  5 %  6 %\nSupplemental NIKE Brand Revenues Details:\nNIKE Brand Revenues by:\nSales to Wholesale Customers $ 27,397 $ 25,608  7 %  14 % $ 25,898  -1 %  -1 %\nSales through NIKE Direct  21,308  18,726  14 %  20 %  16,370  14 %  15 %',
  'Greater China 

In [219]:
queries = query

In [ ]:
rag_queries = [query for query in queries]